# SE446 - Week 4B: HiveQL on the Hadoop Cluster

## 📚 Learning Objectives

By the end of this notebook, you will be able to:
1. Connect to HiveServer2 using Beeline
2. Understand the Hive Metastore (before/after table creation)
3. Create external Hive tables over HDFS data
4. Run analytical HiveQL queries on real crime data
5. Create and execute `.hql` script files (professional practice)
6. Compare the HiveQL approach with MapReduce (Milestone 1)

---

## ⚠️ Important: This Notebook Uses Terminal Commands

Unlike Notebook 4A (which ran everything in Python), this notebook guides you through running **real HiveQL commands on the Hadoop cluster** via your terminal.

### How to follow along

1. **Open a terminal** (separate from this notebook)
2. **SSH into the cluster** using the commands below
3. **Run each HiveQL command** shown in the code cells
4. **Paste your results** into the answer cells provided

The code cells below show you the exact commands to type. **Do NOT run them in this notebook** — run them in your terminal.

---

## 1. Prerequisites ✅

Before starting, make sure you have:

- [ ] SSH access to the cluster (same credentials from Milestone 1)
- [ ] Completed Notebook 4A (HiveQL Emulator)
- [ ] The Chicago crime dataset is already on HDFS at `/data/chicago_crimes.csv`

### Dataset Reminder

This is the **same Chicago crime dataset** you used in Milestone 1 for MapReduce.

| File | Path | Size |
|------|------|------|
| Full dataset | `/data/chicago_crimes.csv` | 8M+ rows |
| Sample | `/data/chicago_crimes_sample.csv` | 10K rows |

**Schema**: `ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, ...`

| Index | Column | Example |
|-------|--------|---------|
| 0 | ID | 10224738 |
| 1 | Case Number | HY411648 |
| 2 | Date | 09/05/2015 01:30:00 PM |
| 3 | Block | 043XX S WOOD ST |
| 4 | IUCR | 0820 |
| 5 | Primary Type | THEFT |
| 6 | Description | $500 AND UNDER |
| 7 | Location Description | STREET |
| 8 | Arrest | false |
| 9 | Domestic | false |

---

## 2. Step 1: Connect to the Cluster 🖥️

Open your terminal and run:

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  RUN THESE COMMANDS IN YOUR TERMINAL (not in this notebook) ║
# ╚══════════════════════════════════════════════════════════════╝

# Step 1: SSH into the cluster
ssh your_id@134.209.172.50

# Step 2: Source Hadoop environment (REQUIRED every login)
source /etc/profile.d/hadoop.sh

# Step 3: Verify the dataset exists on HDFS
hdfs dfs -ls /data/chicago_crimes*.csv

# Step 4: Check the first 3 lines (to see the header + data)
hdfs dfs -cat /data/chicago_crimes_sample.csv | head -3

### 📝 Paste your output here

After running the `head -3` command above, paste the first 3 lines of the CSV below.

**Your output:**

```
(paste here)
```

---

## 3. Step 2: Connect to Hive with Beeline 🐝

### `hive` vs `beeline -u 'jdbc:hive2://'` — What's the Difference?

Both commands start Beeline, but they behave differently:

| | `hive` | `beeline -u 'jdbc:hive2://'` |
|---|--------|-------------------------------|
| **What it does** | Starts Beeline but does NOT connect | Starts Beeline AND connects immediately |
| **Prompt** | `beeline>` (not connected) | `0: jdbc:hive2://>` (connected) |
| **Run queries?** | ❌ No — "No current connection" error | ✅ Yes — ready immediately |
| **Use case** | Only if you want to connect manually after | Recommended — connect and work in one step |

> ⚠️ **Common Mistake**: Students type `hive`, see the `beeline>` prompt, and try to run queries. This gives "No current connection" errors. **Always use `beeline -u 'jdbc:hive2://'` to connect properly.**

### How to Connect

```bash
# RECOMMENDED: Connect directly (one command, ready to query)
beeline -u 'jdbc:hive2://'

# NOT RECOMMENDED: This starts Beeline but does NOT connect
# hive
# (you would then need to type: !connect jdbc:hive2://)
```

You will see this prompt when connected:
```
Connected to: Apache Hive (version 4.0.1)
0: jdbc:hive2://>
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  RUN THIS IN YOUR TERMINAL                                   ║
# ╚══════════════════════════════════════════════════════════════╝

# CORRECT: Connect to Hive (embedded mode - works for all users)
beeline -u 'jdbc:hive2://'

# Expected output:
# Connected to: Apache Hive (version 4.0.1)
# Driver: Hive JDBC (version 4.0.1)
# 0: jdbc:hive2://>

# ---------------------------------------------------------------
# WRONG: This starts Beeline but does NOT connect!
# hive
# (gives "beeline>" prompt but queries will fail with "No current connection")
# ---------------------------------------------------------------

# To exit Beeline when done:
# !quit

---

## 4. Understanding the Hive Metastore 📦

### What is the Metastore?

The **Hive Metastore** stores metadata about your tables:
- Table names, column names, data types
- Where the data files are located (HDFS path)
- Table properties (delimiters, file format, etc.)

> **Key Concept**: The metastore does NOT store the actual data — it only stores information ABOUT the data. The data stays on HDFS.

### Let's See the Metastore BEFORE Creating a Table

First, let's check what databases and tables exist:

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  RUN THESE IN BEELINE (after connecting)                     ║
# ╚══════════════════════════════════════════════════════════════╝

# Check existing databases
SHOW DATABASES;

# Create your own database (use your student ID)
CREATE DATABASE IF NOT EXISTS your_id_db;
USE your_id_db;

# Check tables BEFORE creating anything
SHOW TABLES;
# Expected output: empty (no tables yet)

### 📝 Metastore BEFORE Creating Table

**`SHOW DATABASES;` output:**
```
default
your_id_db
```

**`SHOW TABLES;` output (in your_id_db):**
```
(empty - no tables yet)
```

> **💡 Observation**: The metastore knows about databases but has no table metadata yet. Let's create a table and see what changes.

---

## 5. Step 3: Create an External Table 🏗️

### Method 1: Paste the Full Query Directly

You can copy and paste the **entire** CREATE TABLE statement at once. Hive will wait for the semicolon (`;`) before executing.

> ⚠️ **Do NOT copy line by line** — paste the whole statement at once!

In [ ]:
-- ╔══════════════════════════════════════════════════════════════╗
-- ║  COPY THIS ENTIRE BLOCK AND PASTE IN BEELINE                ║
-- ╚══════════════════════════════════════════════════════════════╝

CREATE EXTERNAL TABLE chicago_crimes (
    id                   STRING,
    case_number          STRING,
    crime_date           STRING,
    block                STRING,
    iucr                 STRING,
    primary_type         STRING,
    description          STRING,
    location_description STRING,
    arrest               STRING,
    domestic             STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/data/'
TBLPROPERTIES ('skip.header.line.count'='1');

### Metastore AFTER Creating Table

Now check the metastore again:

In [ ]:
-- Check tables AFTER creating
SHOW TABLES;

-- See the table metadata stored in metastore
DESCRIBE chicago_crimes;

-- See detailed metadata (storage location, format, etc.)
DESCRIBE FORMATTED chicago_crimes;

### 📝 Metastore AFTER Creating Table

**`SHOW TABLES;` output:**
```
chicago_crimes
```

**`DESCRIBE chicago_crimes;` output:**
```
col_name              data_type
id                    string
case_number           string
crime_date            string
block                 string
iucr                  string
primary_type          string
description           string
location_description  string
arrest                string
domestic              string
```

> **💡 Key Insight**: The metastore now contains:
> - Table name and column definitions
> - Data types for each column
> - Storage location (`/data/` on HDFS)
> - File format (TEXTFILE) and delimiter (`,`)
> 
> **The actual CSV data stays on HDFS** — the metastore only stores metadata!

---

## 6. Method 2: Using HQL Script Files (Professional Practice) 📄

In real data engineering, you don't type queries manually. Instead, you save queries in `.hql` files and execute them. This is:
- **Repeatable** — run the same script anytime
- **Version-controlled** — save scripts in Git
- **Shareable** — team members can use your scripts
- **Auditable** — you have a record of what ran

### Step 1: Create a .hql Script File

Exit Beeline first (`!quit`), then create a script file:

```bash
nano top_crimes.hql
```

Paste the following contents into `top_crimes.hql`:

```sql
-- Top 10 Crime Types Query
-- Make sure you are using your database
USE your_id_db;

SELECT primary_type, COUNT(*) AS cnt
FROM chicago_crimes
GROUP BY primary_type
ORDER BY cnt DESC
LIMIT 10;
```

Save and exit nano:
- **Save**: `CTRL+O`, press Enter
- **Exit**: `CTRL+X`

In [ ]:
-- Top 10 Crime Types Query
-- Make sure you are using your database
USE default;

SELECT primary_type, COUNT(*) AS cnt
FROM chicago_crimes
GROUP BY primary_type
ORDER BY cnt DESC
LIMIT 10;

### Step 2: Execute the HQL Script

There are **3 ways** to run a `.hql` file:

```bash
# Method A: Run from Linux command line (non-interactive)
beeline -u 'jdbc:hive2://' -f top_crimes.hql

# Method B: Run from inside Beeline using SOURCE
beeline -u 'jdbc:hive2://'
# Then at the 0: jdbc:hive2://> prompt:
SOURCE top_crimes.hql;

# Method C: Run a single query inline
beeline -u 'jdbc:hive2://' -e "SELECT COUNT(*) FROM your_id_db.chicago_crimes;"
```

> **Best Practice**: Save all your queries in `.hql` files. Submit scripts in assignments, not screenshots!

### Step 2: Execute the HQL Script

There are **3 ways** to run a `.hql` file:

```bash
# Method A: Run from Linux command line (non-interactive)
beeline -u jdbc:hive2://localhost:10000 -f top_crimes.hql

# Method B: Run from inside Beeline using SOURCE
beeline -u jdbc:hive2://localhost:10000
# Then at the 0: jdbc:hive2://localhost:10000> prompt:
SOURCE top_crimes.hql;

# Method C: Run with -e flag for single queries
beeline -u jdbc:hive2://localhost:10000 -e "SELECT COUNT(*) FROM your_id_db.chicago_crimes;"
```

> **Best Practice**: Save all your queries in `.hql` files. Submit scripts in assignments, not screenshots!

---

## 7. Quick Sanity Check ✅

Verify the table is working with a simple query:

In [ ]:
#Run in Beeline after connecting
SELECT * FROM chicago_crimes LIMIT 5;

#Count total rows (launches a MapReduce job!)
SELECT COUNT(*) FROM chicago_crimes;

### 📝 Paste your output here

**`SELECT * FROM chicago_crimes LIMIT 5;` output:**

```
(paste here)
```

**`SELECT COUNT(*) FROM chicago_crimes;` output:**

```
Total rows: (paste the number here)
```

**How long did the COUNT query take?** ______ seconds

> **💡 Notice**: Even a simple `COUNT(*)` launches a distributed MapReduce job! You'll see progress bars in your terminal. This is the power of Hive — it automatically distributes the work across the cluster.

---

## 8. Query 1: Crime Type Distribution 📊

**Question**: What are the top 10 most common crime types in Chicago?

This is the same question from **Milestone 1 (Task 2)** where you wrote MapReduce code. Now do it in one SQL query:

-- Query 1: Top 10 crime types
SELECT primary_type, COUNT(*) AS cnt
FROM chicago_crimes
GROUP BY primary_type
ORDER BY cnt DESC
LIMIT 10;

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  RUN THIS IN THE HIVE SHELL (hive>)                          ║
# ╚══════════════════════════════════════════════════════════════╝

# Exit the Hive shell
# quit;

# Exit SSH
# exit

---

## 9. Reflection: MapReduce vs. HiveQL 🤔

You've now solved the **same problems** using two different approaches. Let's compare them.

### Comparison Table

| Aspect | MapReduce (Milestone 1) | HiveQL (This Notebook) |
|--------|------------------------|------------------------|
| **Code needed** | `mapper.py` + `reducer.py` (2 files) | 1 SQL query |
| **Lines of code** | ~20-30 lines | ~5-8 lines |
| **Skill required** | Python programming | SQL knowledge |
| **Flexibility** | Can do any computation | Limited to SQL operations |
| **Ease of use** | Harder (manual key-value logic) | Easier (declarative) |
| **Under the hood** | You write the MapReduce logic | Hive generates it for you |

---

## 11. Summary & Quick Reference

### What you learned today

1. **Connect to Hive** using `beeline -u 'jdbc:hive2://'`
2. **Metastore** stores table metadata (schema, location), not actual data
3. **External tables** point to existing HDFS data — no data copying
4. **One SQL query** replaces mapper + reducer + streaming command
5. **HQL files** are the professional way to work with Hive

### Quick Reference Commands

| Task | Command |
|------|---------|
| Connect to Hive | `beeline -u 'jdbc:hive2://'` |
| Run HQL file | `beeline -u 'jdbc:hive2://' -f script.hql` |
| Run single query | `beeline -u 'jdbc:hive2://' -e "SELECT..."` |
| Exit Beeline | `!quit` |

### `hive` vs `beeline -u 'jdbc:hive2://'`

| Command | Connects? | Can run queries? |
|---------|-----------|-----------------|
| `hive` | ❌ No | ❌ "No current connection" |
| `beeline -u 'jdbc:hive2://'` | ✅ Yes | ✅ Immediately |

### Key Takeaway

> **Hive = SQL + Distributed Computing**
>
> Write SQL → Hive generates MapReduce → Cluster processes data → Results returned

---

## 10. Summary

### What you learned today

1. **Hive provides SQL on Hadoop** — you can query massive HDFS datasets without writing MapReduce code
2. **External tables** point to existing HDFS data — no data movement needed
3. **One SQL query** replaces mapper + reducer + streaming command
4. **Under the hood**, Hive still uses MapReduce/Tez — you can see the jobs running

### Key takeaway

> **Hive gives you the productivity of SQL with the power of distributed computing.**
> 
> You don't have to choose — Hive translates your SQL into MapReduce/Tez jobs automatically.